In [3]:
# Importa el módulo 'random' para generar números aleatorios.
import random

# Genetic Algorithm parameters
# Define el tamaño de la población (cantidad de soluciones candidatas).
POPULATION_SIZE = 50
# Define la tasa de mutación (probabilidad de que un individuo mute).
MUTATION_RATE = 0.1
# Define el número máximo de generaciones o iteraciones del algoritmo.
MAX_GENERATIONS = 100

# Generate a random board state
# Función para crear un estado de tablero inicial aleatorio (un individuo/cromosoma).
def generate_board_state():
    # Crea una lista de 8 números (0 a 7), donde el índice es la columna y el valor es la fila de la reina.
    board_state = [random.randint(0, 7) for _ in range(8)]
    return board_state

# Calculate the fitness of a board state
# Función para calcular la aptitud (fitness) de un estado del tablero.
def calculate_fitness(board_state):
    # Inicializa el contador de conflictos (reinas atacándose mutuamente).
    conflicts = 0
    # Bucle anidado para comparar cada par de reinas.
    for i in range(8):
        for j in range(i + 1, 8):
            # Comprueba conflicto en la misma fila (board_state[i] == board_state[j]) o en diagonal.
            # abs(board_state[i] - board_state[j]) == j - i verifica conflicto diagonal.
            if board_state[i] == board_state[j] or abs(board_state[i] - board_state[j]) == j - i:
                conflicts += 1
    # La aptitud se define como el máximo de pares no conflictivos (28) menos los conflictos actuales.
    return 28 - conflicts  # Max fitness = 28 (no conflicts)

# Select parents for crossover using tournament selection
# Función para seleccionar un padre usando el método de selección por torneo.
def tournament_selection(population):
    # Define el tamaño del subconjunto (torneo).
    tournament_size = 5
    # Selecciona un subconjunto aleatorio de la población.
    tournament = random.sample(population, tournament_size)
    # Devuelve el individuo con el mayor fitness (el ganador del torneo).
    return max(tournament, key=lambda x: x[1])

# Crossover operation (single-point crossover)
# Función para realizar el cruce (crossover) de un solo punto.
def crossover(parent1, parent2):
    # Elige un punto de corte aleatorio.
    crossover_point = random.randint(1, 7)
    # Crea el hijo combinando la primera parte del padre1 y la segunda del padre2.
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Mutation operation (swap two positions)
# Función para realizar la operación de mutación (intercambio de dos posiciones/filas).
def mutate(board_state):
    # Elige dos posiciones (columnas) aleatorias y distintas.
    pos1, pos2 = random.sample(range(8), 2)
    # Intercambia los valores de las filas de las reinas en esas dos posiciones.
    board_state[pos1], board_state[pos2] = board_state[pos2], board_state[pos1]
    return board_state

# Generate the initial population
# Genera la población inicial: tuplas (estado_tablero, fitness=0).
population = [(generate_board_state(), 0) for _ in range(POPULATION_SIZE)]

# Main Genetic Algorithm loop
# Bucle principal que itera a través de las generaciones.
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    # Recalcula y actualiza el fitness real de cada individuo en la población.
    population = [(board_state, calculate_fitness(board_state)) for board_state, _ in population]

    # Check if solution is found
    # Encuentra el mejor estado de tablero de la generación actual.
    best_board_state = max(population, key=lambda x: x[1])[0]
    # Comprueba si el fitness máximo (28) ha sido alcanzado.
    if calculate_fitness(best_board_state) == 28:
        # Si se encuentra la solución, la imprime y sale del bucle.
        print("Solution found in generation", generation)
        break

    # Create the next generation
    # Inicializa la lista que contendrá a la nueva población.
    new_population = []

    # Elitism: Keep the best board state from the previous generation
    # Aplica elitismo: añade al mejor individuo (el más apto) directamente a la nueva generación.
    new_population.append(max(population, key=lambda x: x[1]))

    # Perform selection, crossover, and mutation
    # Bucle para generar el resto de la nueva población hasta alcanzar el tamaño definido.
    while len(new_population) < POPULATION_SIZE:
        # Selecciona dos padres usando el torneo.
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        # Realiza el cruce para crear un hijo a partir de los estados de los padres.
        child = crossover(parent1[0], parent2[0])
        # Comprueba si la mutación debe ocurrir (probabilidad MUTATION_RATE).
        if random.random() < MUTATION_RATE:
            # Aplica la mutación al hijo.
            child = mutate(child)
        # Añade el nuevo hijo a la nueva población (con fitness temporal 0).
        new_population.append((child, 0))

    # Update the population
    # La nueva población reemplaza a la vieja para la siguiente iteración.
    population = new_population

# Print the best solution
# Imprime el mejor estado de tablero encontrado (la solución o el mejor intento).
print("Best solution:", best_board_state)

Solution found in generation 11
Best solution: [1, 7, 5, 0, 2, 4, 6, 3]


In [11]:
# Select parents for crossover using roulette wheel selection selection
# Define la función para seleccionar un individuo (padre) usando el método de la Ruleta.
def roulette_wheel_selection(population):
    # Calculate total fitness sum of the population
    # Calcula la suma total de los valores de fitness (aptitud) de todos los individuos en la población.
    # El fitness de cada individuo está en la posición [1] de la tupla (estado, fitness).
    total_fitness = sum(individual[1] for individual in population)

    # Generate a random number between 0 and the total fitness sum
    # Genera un número aleatorio ('spin') entre 0 y el fitness total. Esto simula el "giro" de la ruleta.
    spin = random.uniform(0, total_fitness)

    # Iterate through the population and sum fitness values until the spin value is reached
    # Inicializa el acumulador de fitness.
    fitness_sum = 0
    # Itera sobre cada individuo de la población.
    for individual in population:
        # Suma el fitness del individuo actual al acumulador.
        fitness_sum += individual[1]
        # Comprueba si la suma acumulada ha alcanzado o superado el valor aleatorio del 'spin'.
        if fitness_sum >= spin:
            # Si se supera, el individuo actual es seleccionado como el padre y se devuelve.
            # Los individuos con mayor fitness tienen un segmento más grande y son más propensos a ser seleccionados.
            return individual

In [12]:
# ====================================================================================
# --- 1. CONFIGURACIÓN E INICIALIZACIÓN ---
# ====================================================================================

# Importa el módulo 'random' para generar números aleatorios.
import random

# Genetic Algorithm parameters
# Define el tamaño de la población
POPULATION_SIZE = 50
# Define la tasa de mutación
MUTATION_RATE = 0.1
# Define el número máximo de generaciones 
MAX_GENERATIONS = 100

# Generate the initial population
# Genera la población inicial: tuplas (estado_tablero, fitness=0).
population = [(generate_board_state(), 0) for _ in range(POPULATION_SIZE)]


# ====================================================================================
# --- 2. FUNCIONES DEL ALGORITMO GENÉTICO ---
# ====================================================================================

# Generate a random board state
# Función para crear un estado de tablero inicial aleatorio (un individuo/cromosoma).
# TABLERO:
def generate_board_state():
    # Crea una lista de 8 números (0 a 7), donde el índice es la columna y el valor es la fila de la reina.
    board_state = [random.randint(0, 7) for _ in range(8)]
    return board_state

# Calculate the fitness of a board state (TABLERO)
# Función para calcular la aptitud (fitness) de un estado del tablero.
def calculate_fitness(board_state):
    # Inicializa el contador de conflictos (reinas atacándose mutuamente).
    conflicts = 0
    # Bucle anidado para comparar cada par de reinas.
    for i in range(8):
        for j in range(i + 1, 8):
            # Comprueba conflicto en la misma fila o en diagonal.
            if board_state[i] == board_state[j] or abs(board_state[i] - board_state[j]) == j - i:
                conflicts += 1
    # La aptitud máxima es 28 (pares posibles sin conflicto).
    return 28 - conflicts  # Max fitness = 28 (no conflicts)

# Select parents for crossover using "tournament selection"
# Función para seleccionar un padre usando el método de selección por torneo.
def tournament_selection(population):
    # Define el tamaño del subconjunto (torneo).
    tournament_size = 5
    # Selecciona un subconjunto aleatorio de la población.
    tournament = random.sample(population, tournament_size)
    # Devuelve el individuo con el mayor fitness (el ganador del torneo).
    return max(tournament, key=lambda x: x[1])

# Crossover operation (single-point crossover)
# Función para realizar el cruce (crossover) de un solo punto.
def crossover(parent1, parent2):
    # Elige un punto de corte aleatorio.
    crossover_point = random.randint(1, 7)
    # Crea el hijo combinando la primera parte del padre1 y la segunda del padre2.
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Mutation operation (swap two positions)
# Función para realizar la operación de mutación (intercambio de dos posiciones/filas).
def mutate(board_state):
    # Elige dos posiciones (columnas) aleatorias y distintas.
    pos1, pos2 = random.sample(range(8), 2)
    # Intercambia los valores de las filas de las reinas en esas dos posiciones.
    board_state[pos1], board_state[pos2] = board_state[pos2], board_state[pos1]
    return board_state


# ====================================================================================
# --- 3. EJECUCIÓN DEL BUCLE PRINCIPAL ---
# ====================================================================================

# Main Genetic Algorithm loop
# Bucle principal que itera a través de las generaciones.
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    # Recalcula y actualiza el fitness real de cada individuo en la población.
    population = [(board_state, calculate_fitness(board_state)) for board_state, _ in population]

    # Check if solution is found
    # Encuentra el mejor estado de tablero de la generación actual.
    best_board_state = max(population, key=lambda x: x[1])[0]
    # Comprueba si el fitness máximo (28) ha sido alcanzado.
    if calculate_fitness(best_board_state) == 28:
        # Si se encuentra la solución, la imprime y sale del bucle.
        print("Solution found in generation", generation)
        break

    # Create the next generation
    # Inicializa la lista que contendrá a la nueva población.
    new_population = []

    # Elitism: Keep the best board state from the previous generation
    # Aplica elitismo: añade al mejor individuo (el más apto) directamente a la nueva generación.
    new_population.append(max(population, key=lambda x: x[1]))

    # Perform selection, crossover, and mutation
    # Bucle para generar el resto de la nueva población.
    while len(new_population) < POPULATION_SIZE:
        # Selecciona dos padres usando el torneo.
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        # Realiza el cruce para crear un hijo.
        child = crossover(parent1[0], parent2[0])
        # Comprueba si la mutación debe ocurrir.
        if random.random() < MUTATION_RATE:
            # Aplica la mutación.
            child = mutate(child)
        # Añade el nuevo hijo a la nueva población.
        new_population.append((child, 0))

    # Update the population
    # La nueva población reemplaza a la vieja para la siguiente iteración.
    population = new_population

# Print the best solution
# Imprime el mejor estado de tablero encontrado.
print("Best solution:", best_board_state)

Best solution: [4, 6, 1, 0, 2, 5, 7, 1]


In [18]:
import random

# Select parents for crossover using roulette wheel selection selection
def roulette_wheel_selection(population):
    # Calculate total fitness sum of the population
    total_fitness = sum(individual[1] for individual in population)

    # Generate a random number between 0 and the total fitness sum
    spin = random.uniform(0, total_fitness)

    # Iterate through the population and sum fitness values until the spin value is reached
    fitness_sum = 0
    for individual in population:
        fitness_sum += individual[1]
        if fitness_sum >= spin:
            return individual

In [19]:
import random

# Genetic Algorithm parameters
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
MAX_GENERATIONS = 100

# Generate a random board state
def generate_board_state():
    board_state = [random.randint(0, 7) for _ in range(8)]
    return board_state

# Calculate the fitness of a board state
def calculate_fitness(board_state):
    conflicts = 0
    for i in range(8):
        for j in range(i + 1, 8):
            if board_state[i] == board_state[j] or abs(board_state[i] - board_state[j]) == j - i:
                conflicts += 1
    return 28 - conflicts  # Max fitness = 28 (no conflicts)

# Select parents for crossover using tournament selection
def tournament_selection(population):
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x[1])

# Crossover operation (single-point crossover)
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 7)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Mutation operation (swap two positions)
def mutate(board_state):
    pos1, pos2 = random.sample(range(8), 2)
    board_state[pos1], board_state[pos2] = board_state[pos2], board_state[pos1]
    return board_state

# Generate the initial population
population = [(generate_board_state(), 0) for _ in range(POPULATION_SIZE)]

# Main Genetic Algorithm loop
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    population = [(board_state, calculate_fitness(board_state)) for board_state, _ in population]

    # Check if solution is found
    best_board_state = max(population, key=lambda x: x[1])[0]
    if calculate_fitness(best_board_state) == 28:
        print("Solution found in generation", generation)
        break

    # Create the next generation
    new_population = []

    # Elitism: Keep the best board state from the previous generation
    new_population.append(max(population, key=lambda x: x[1]))

    # Perform selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        child = crossover(parent1[0], parent2[0])
        if random.random() < MUTATION_RATE:
            child = mutate(child)
        new_population.append((child, 0))

    # Update the population
    population = new_population

# Print the best solution
print("Best solution:", best_board_state)

Best solution: [1, 3, 0, 7, 5, 0, 2, 6]


In [20]:
import random

# Select parents for crossover using roulette wheel selection selection
def roulette_wheel_selection(population):
    # Calculate total fitness sum of the population
    total_fitness = sum(individual[1] for individual in population)

    # Generate a random number between 0 and the total fitness sum
    spin = random.uniform(0, total_fitness)

    # Iterate through the population and sum fitness values until the spin value is reached
    fitness_sum = 0
    for individual in population:
        fitness_sum += individual[1]
        if fitness_sum >= spin:
            return individual